In [1]:
import torch
from datafeed import RegressionDataset, ClassificationDataset
import torch.nn as nn
import torch.optim as optim
import math 
from model import FuncControlModel
import copy
# Define your model
import random 
# data is sinx , x is 0 to 1
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
from tqdm import tqdm 
from torchmetrics import MeanAbsolutePercentageError
import datetime
from functions import FUNC_CLASSES
from func_manage import transfer_methods
from functions import *
import torchvision 
import torchvision.transforms as transforms
from func_prune import get_topk_funcs, remove_other_funcs
import inspect
from mixedfunc import MixedFunc

/home/bosung/anaconda3/lib/python3.9/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/bosung/anaconda3/lib/python3.9/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Create dynamic Funcpool. 

class_keys = list(FUNC_CLASSES.keys())
classes = [FUNC_CLASSES[key] for key in class_keys]

# get initial func pool method list
flist = [dir(FuncPool)[i] for i in range(len(dir(FuncPool))) if not dir(FuncPool)[i].startswith('_')]
print("Initial len: ", len(flist))
print("Initial flist: ", flist)

# transfer methods from classes to FuncPool
transfer_methods(classes, FuncPool)

# get final func pool method list
flist = [dir(FuncPool)[i] for i in range(len(dir(FuncPool))) if not dir(FuncPool)[i].startswith('_')]
print("Transfer_res : ", len(flist))
print("Transfered flist: ", flist)
        

Initial len:  7
Initial flist:  ['add', 'cos', 'mean', 'relu', 'sigmoid', 'sin', 'sub']
Transfer_res :  29
Transfered flist:  ['abs', 'add', 'ceil', 'clamp', 'clip', 'cos', 'equal', 'floor', 'frac', 'greater', 'greater_equal', 'less', 'less_equal', 'logical_and', 'logical_not', 'logical_or', 'logical_xor', 'maximum', 'mean', 'minimum', 'neg', 'not_equal', 'relu', 'round', 'sigmoid', 'sign', 'sin', 'sub', 'trunc']


In [3]:
def func_inspect_func(func, num_args, testmodule):
    # implement simple loop to test stability of function
    # test 4 cases at base function, 4 cases at derived function
    # if all cases are correct, then return True
    # else return False
    TESTLENGTH = 100

    min_val = -1e10
    max_val = 1e10
    min_mag = 1e-10
    neg_min_mag = -1e-10
    device = "cuda:0"
    check_res = {"base_0": True, "base_1": True, "derived_0": True, "derived_1": True}
    # check if function makes nan or inf with torch
    
    optimizer = optim.SGD(testmodule.parameters(), lr=0.01)
    for iter in tqdm(range(TESTLENGTH)):
        x = random.uniform(min_val, max_val)
        x = torch.tensor(x, requires_grad=True).float().to(device)
        corrected_args = torch.Tensor([[x for i in range(num_args)]]).to(device)

        try:
            res = testmodule(func(corrected_args))
        except:
            print(corrected_args)
            print(corrected_args.shape)
            break
        
 

        # check if any element is nan or inf
        if torch.isnan(*res).any() or torch.isinf(*res).any():
            print("Nan or Inf detected at base function, iter : ", iter)
            check_res["base_0"] = False
        
        x_little = random.uniform(neg_min_mag, min_mag)
        x_little = torch.tensor(x_little, requires_grad=True).float().to(device)
        x_little_corrected = torch.Tensor([x_little for i in range(num_args)]).to(device)
        res_little = testmodule(func(x_little_corrected))


        # check if any element is nan or inf
        if torch.isnan(res_little).any() or torch.isinf(res_little).any():
            print("Nan or Inf detected at base function, iter : ", iter)
            check_res["base_1"] = False
            # check if derivative is nan or inf
            
        res.backward()
        param = [x for x in testmodule.parameters()]
        if torch.isnan(param[0].grad).any() or torch.isinf(param[0].grad).any():
            print("Nan or Inf detected at derived function, iter : ", iter)
            check_res["derived_0"] = False

        optimizer.zero_grad()
        # check if derivative is nan or inf
        res_little.backward()
        if torch.isnan(param[0].grad).any() or torch.isinf(param[0].grad).any():
            print("Nan or Inf detected at derivced function, iter : ", iter)
            check_res["derived_1"] = False
        
    # test base function
    return check_res

In [4]:
# create mixedop instance 
target_mix = MixedFunc(FuncPool())


# you can access number of arguments by MixedFunc.num_args.
result_dict = {}
for i in range(len(target_mix.flist)):
    func_name = target_mix.flist[i]
    func = getattr(target_mix.Flist, func_name)
    alph_sim = nn.Linear(target_mix.num_args[i],1).to("cuda:0")

    inspect_res = func_inspect_func(func, target_mix.num_args[i], alph_sim)
    result_dict[func_name] = inspect_res
    
    # if any values are false, return it
    for key in result_dict.keys():
        if result_dict[key] == False:
            print(func_name)
    

100%|██████████| 100/100 [00:00<00:00, 795.37it/s]


{'abs': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}}


  0%|          | 0/100 [00:00<?, ?it/s]


tensor([[8.1289e+09, 8.1289e+09]], device='cuda:0')
torch.Size([1, 2])
{'abs': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'add': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}}


100%|██████████| 100/100 [00:00<00:00, 964.87it/s]


...

{'abs': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'add': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'ceil': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'clamp': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'clip': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'cos': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'equal': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'floor': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'frac': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'greater': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'greater_equal': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'less': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'less_equal': {'base_0': Tr

  0%|          | 0/100 [00:00<?, ?it/s]


tensor([[6.9973e+09, 6.9973e+09]], device='cuda:0')
torch.Size([1, 2])
{'abs': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'add': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'ceil': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'clamp': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'clip': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'cos': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'equal': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'floor': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'frac': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'greater': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'greater_equal': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'less': {'base_0': True, 'base_1': T

100%|██████████| 100/100 [00:00<00:00, 1111.81it/s]

{'abs': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'add': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'ceil': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'clamp': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'clip': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'cos': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'equal': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'floor': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'frac': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'greater': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'greater_equal': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'less': {'base_0': True, 'base_1': True, 'derived_0': True, 'derived_1': True}, 'less_equal': {'base_0': Tr